# 3: Extract Local Weighting

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle
import time

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler

from collections import Counter

from copy import deepcopy

import lime
import lime.lime_tabular

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation
from keras import backend as K

# For Deep Learning Explanations
from deepexplain.tensorflow import DeepExplain
import deeplift
from deeplift.conversion import kerasapi_conversion as kc

Using TensorFlow backend.


In [2]:
df = pd.read_csv("processed_df.csv")

In [3]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [4]:
model = load_model('NN.h5')

## First Generate Gradient & Significance LIME Matrices:

In [5]:
def flatten_predict(i):
    """
    LIME doesn't support the format Keras uses, so we need a small helper function
    
    This just predicts a probability and gives the alternative also.
    """
    
    global model
    
    idx = model.predict_classes(i)
    probs = model.predict_proba(i)
    idx = np.array([idx]).T
    
    probability_yes = probs[np.arange(probs.shape[0])[:, None], idx]
    
    x = np.zeros((probability_yes.shape[0], 1))
    probability_no = (x + 1) - probability_yes
    final = np.append(probability_no, probability_yes, axis=1)
    
    return final

In [6]:
def which_features_ohe(df):
    """
    Locate which features are categorical ohe, they should just have 2 values 0 and 1
    
    return: a list of integers referring to the ohe indexes
    """
    
    categorical_features = list()
    for i in range(len(df.columns)):
        if df[df.columns[i]].value_counts().shape == (2,):
            categorical_features.append(i)
    return categorical_features

In [7]:
categorical_features = which_features_ohe(df)
feature_names = df.columns
num_features = df.shape[1]

In [8]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, 
                                                   feature_names=feature_names, 
                                                   categorical_features=categorical_features,
                                                   class_names=["0", "1", "2"],
                                                   verbose=False, 
                                                   discretize_continuous=False,
                                                   mode='classification')

In [9]:
X_train_grad_LIME = list()

for i in range(len(X_train)):
    
    if i % 1000 == 0:
        print((i / len(X_train)) * 100, "% done...")
    
    qc = X_train[i]
    exp = explainer.explain_instance(qc, flatten_predict, num_features=num_features)
    
    # Get real coefficients
    coefs = exp.as_map()[1]
    coefs.sort()
    coefs = [x[1] for x in coefs]

    X_train_grad_LIME.append(np.append(coefs, exp.intercept[1]).tolist())

0.0 % done...
1.644736842105263 % done...
3.289473684210526 % done...
4.934210526315789 % done...
6.578947368421052 % done...
8.223684210526317 % done...
9.868421052631579 % done...
11.513157894736842 % done...
13.157894736842104 % done...
14.802631578947366 % done...
16.447368421052634 % done...
18.092105263157894 % done...
19.736842105263158 % done...
21.38157894736842 % done...
23.026315789473685 % done...
24.671052631578945 % done...
26.31578947368421 % done...
27.960526315789476 % done...
29.605263157894733 % done...
31.25 % done...
32.89473684210527 % done...
34.53947368421053 % done...
36.18421052631579 % done...
37.82894736842105 % done...
39.473684210526315 % done...
41.118421052631575 % done...
42.76315789473684 % done...
44.40789473684211 % done...
46.05263157894737 % done...
47.69736842105263 % done...
49.34210526315789 % done...
50.98684210526315 % done...
52.63157894736842 % done...
54.276315789473685 % done...
55.92105263157895 % done...
57.56578947368421 % done...
59.21

In [10]:
X_test_grad_LIME = list()

for i in range(len(X_test)):
    
    if i % 1000 == 0:
        print((i / len(X_test)) * 100, "% done...")
    
    qc = X_test[i]
    exp = explainer.explain_instance(qc, flatten_predict, num_features=num_features)
    
    # Get real coefficients
    coefs = exp.as_map()[1]
    coefs.sort()
    coefs = [x[1] for x in coefs]

    X_test_grad_LIME.append(np.append(coefs, exp.intercept[1]).tolist())

0.0 % done...
14.801657785671996 % done...
29.60331557134399 % done...
44.40497335701599 % done...
59.20663114268798 % done...
74.00828892835997 % done...
88.80994671403198 % done...


In [11]:
X_train_grad_LIME = np.array(X_train_grad_LIME)
X_test_grad_LIME = np.array(X_test_grad_LIME)
np.save("X_train_grad_LIME", X_train_grad_LIME)
np.save("X_test_grad_LIME", X_test_grad_LIME)

## Integrated Gradients and Layerwise Relevance Propagation
https://github.com/marcoancona/DeepExplain

In [12]:
from keras.utils import to_categorical

nn_preds_train = model.predict_classes(X_train)
nn_preds_test = model.predict_classes(X_test)
oh_y_train = to_categorical(nn_preds_train)
oh_y_test = to_categorical(nn_preds_test)

In [13]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    xs = X_train
    
    X_train_intgrad = de.explain('intgrad', target_tensor * oh_y_train, input_tensor, xs)
    X_train_lrp = de.explain('elrp', target_tensor * oh_y_train, input_tensor, xs)
    
    
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    xs = X_test
    
    X_test_intgrad = de.explain('intgrad', target_tensor * oh_y_test, input_tensor, xs)
    X_test_lrp = de.explain('elrp', target_tensor * oh_y_test, input_tensor, xs)
    
# Save the integrated gradients and LRP 
np.save("X_train_intgrad", X_train_intgrad)
np.save("X_test_intgrad", X_test_intgrad)
np.save("X_train_lrp", X_train_lrp)
np.save("X_test_lrp", X_test_lrp)

DeepExplain: running "intgrad" explanation method (3)
Model with multiple inputs:  False
DeepExplain: running "elrp" explanation method (4)
Model with multiple inputs:  False
DeepExplain: running "intgrad" explanation method (3)
Model with multiple inputs:  False
DeepExplain: running "elrp" explanation method (4)
Model with multiple inputs:  False


## DeepLIFT Contributions:
https://github.com/kundajelab/deeplift

In [14]:
deeplift_model =\
    kc.convert_model_from_saved_files(
        "NN.h5",
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault) 
    
find_scores_layer_idx = 0

deeplift_contribs_func = deeplift_model.get_target_contribs_func(
                            find_scores_layer_idx=find_scores_layer_idx,
                            target_layer_idx=-2)

X_train_deeplift = list()
X_test_deeplift = list()

for i in range(model.get_weights()[-1].shape[0]):
    
    train = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_train],
                                             batch_size=10,
                                             progress_update=1000))

    test = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_test],
                                             batch_size=10,
                                             progress_update=1000))
    
    X_train_deeplift.append(train)
    X_test_deeplift.append(test)
    
X_train_dl = np.array(X_train_deeplift)
X_test_dl = np.array(X_test_deeplift)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
Heads-up: I assume softmax is the output layer, not an intermediate one; if it's an intermediate layer, please let me know and I will prioritise that use-case
For layer 3 the preceding linear layer is 2 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
No reference provided - using zeros
Done 0
Done 1000
Done 2000
Done 3000
Done 4000
Done 5000
Done 6000
Done 7000
Done 8000
Done 9000
Done 10000
Done 11000
Done 12000
Done 13000
Done 14000
Done 15000
Done 16000
Done 17000
Done 18000
Done 19000
Done 20000
Done 21000
Done 22000
Done 23000
Done 24000
Done 25000
Done 26000
Done 27000
Done 28000
Done 29000
Done 30000
Done 31000
Done 32000
Done 33000
Done 34000
Done 35000
Done 36000
Done 

In [15]:
X_train_deeplift = list()
X_test_deeplift = list()

for i in range(len(nn_preds_train)):
    index = nn_preds_train[i]
    X_train_deeplift.append(X_train_dl[index][i])

for i in range(len(nn_preds_test)):
    index = nn_preds_test[i]
    X_test_deeplift.append(X_test_dl[index][i])

In [16]:
X_train_deeplift = np.array(X_train_deeplift)
X_test_deeplift = np.array(X_test_deeplift)

np.save("X_train_deeplift", X_train_deeplift)
np.save("X_test_deeplift", X_test_deeplift)